In [ ]:
using Plots
using HDF5
using FFTW
using Statistics
using LaTeXStrings
using StatsBase
using LinearAlgebra

In [ ]:
fname="/local/scratch/public/cjh225/v2_peta4/Feb2019/P_force/t8/start.h5"

In [ ]:
# using Profile
# @profile out_analysis.out2chi(fname)
# typeof(chi)

Re, Ri_t, Pr = 1e4, 1, 1

f = h5open(fname,"r")
(NX,NY,NZ) = size(f["TH1"])
S1 = read(f["TH1"])
close(f)
println("Loaded TH variable")

# chi = zeros(NX,NY,NZ)

N2 = zeros(NX,NY,NZ)

for k=1:NZ
    tmp = [S1[:,end-31:end,k] S1[:,:,k] S1[:,1:32,k]]
    N2[:,:,k] = (tmp[:,65:end] - tmp[:,1:end-64]) ./ (64*2*pi/NY) .+ 1
end
println("Computed finite difference N²")

# Sme = mean(S1,dims=(1,3))
# S1 = S1.-Sme

FFTW.set_num_threads(32)
ft1 = plan_rfft(S1[:,:,1],1)
ft2 = plan_rfft(S1[:,:,1],2)
ft3 = plan_rfft(S1[:,1,:],2)

CIKX=im*(0:NX÷2)
CIKY=reshape(im*(0:NY÷2),(1,NY÷2+1))
CIKZ=reshape(im*(0:NZ÷2),(1,NZ÷2+1))

# for k=1:NZ
#     CS1 = ft1 * S1[:,:,k]
#     CS1 = CIKX.*CS1
#     chi[:,:,k] += (inv(ft1) * CS1).^2
# end
# println("Computed 1st derivative")

# for k=1:NZ
#     CS1 = ft2 * S1[:,:,k]
#     CS1 = CIKY.*CS1
#     chi[:,:,k] += (inv(ft2) * CS1).^2
# end
# println("Computed 2nd derivative")

# for j=1:NY
#     CS1 = ft3 * S1[:,j,:]
#     CS1 = CIKZ.*CS1
#     chi[:,j,:] += (inv(ft3) * CS1).^2
# end
# println("Computed 3rd derivative")

# chi *= Ri_t/Re/Pr

S1 = nothing

GC.gc()

In [ ]:
varinfo()

In [ ]:
dname=("U1","U2","U3")
eps, EK = zeros(NX,NY,NZ), zeros(NX,NY,NZ)

for i=1:3
    f = h5open(fname,"r")
    S1 = read(f[dname[i]])
    close(f)
    println("Loaded U$(i) variable")
    
    if mod(i,2)==1
        Sme = mean(S1,dims=(1,3))
        S1 = S1.-Sme
    end
    EK += 0.5*S1.^2
    println("TKE computed")

    for k=1:NZ
        CS1 = ft1 * S1[:,:,k]
        CS1 = CIKX.*CS1
        eps[:,:,k] += (inv(ft1) * CS1).^2
    end
    println("Computed 1st derivative")

    for k=1:NZ
        CS1 = ft2 * S1[:,:,k]
        CS1 = CIKY.*CS1
        eps[:,:,k] += (inv(ft2) * CS1).^2
    end
    println("Computed 2nd derivative")

    for j=1:NY
        CS1 = ft3 * S1[:,j,:]
        CS1 = CIKZ.*CS1
        eps[:,j,:] += (inv(ft3) * CS1).^2
    end
    println("Computed 3rd derivative")
end

eps/=Re
S1=nothing

GC.gc()

In [ ]:
# GC.gc()
heatmap(transpose(Fr[:,:,1]),c=:tempo,clim=(0,0.01))
# histogram(Fr[:,:,1][:])
# pgfplots()
# histogram(log10.(chi[:]),bins=-8:0.1:-1)
# size(CS1)
#varinfo()
#backend()


In [ ]:
eps_edges = -8:1/64:-1
chi_edges = -8:1/64:-1

for i=1:NZ
    global G
    h = fit(Histogram, (log10.(eps[:,:,i][:]), log10.(chi[:,:,i][:])), (eps_edges,chi_edges))
    if i==1
        G = h
    else
        merge!(G,h)
    end
end
G = normalize(G, mode=:pdf)

In [ ]:
plotly()
heatmap(G,c=:amp,aspect_ratio=:equal)

In [ ]:
mname = "$(fname[1:end-8])mean.h5"
h5open(mname,"r") do f
    global THme
    THme = read(f["THme/0000"])
end
# size(THme)
fty = plan_rfft(THme,1)
CS1 = fty * THme
CS1 = CIKX .* CS1
N2me = 1 .+ inv(fty) * CS1
N2me = reshape(N2,(1,NY))

In [ ]:
# N2 = N2 .* (N2 .> 0)
Fr = (eps ./ EK).^2 ./ N2
# Gamma = chi ./ eps
for i=1:1024
    for j=1:1024
        for k=1:1024
            if Fr[i,j,k]<=0; Fr[i,j,k]=1e-10; end
        end
    end
end

In [ ]:
Fr_edges = -4:1/64:1
Gamma_edges = -3:1/64:1

for i=1:NZ
    global G2
    h = fit(Histogram, (log10.(Fr[:,:,i][:]), log10.(Gamma[:,:,i][:])), (Fr_edges,Gamma_edges))
    if i==1
        G2 = h
    else
        merge!(G2,h)
    end
end
G2 = normalize(G2, mode=:pdf)

In [ ]:
heatmap(Fr_edges[2:end],Gamma_edges[2:end],transpose(G2.weights),aspect_ratio=:equal,c=:amp)

In [ ]:
eps_edges = -8:1/64:-1
chi_edges = -8:1/64:-1

# varinfo()
Fr_bin = zeros(length(eps_edges)-1,length(chi_edges)-1)
N = zeros(length(eps_edges)-1,length(chi_edges)-1)

for i=1:1024
    for j=1:1024
        for k=1:1024
            if ~isnan(Fr[i,j,k])
                idx = floor(Int, (log10(eps[i,j,k])-eps_edges[1])*64+1)
                jdx = floor(Int, (log10(chi[i,j,k])-chi_edges[1])*64+1)
                idx = min(size(N,1), idx)
                idx = max(1, idx)
                jdx = min(size(N,2), jdx)
                jdx = max(1, jdx)
                Fr_bin[idx,jdx] += log10(Fr[i,j,k])
                N[idx,jdx] += 1
            end
        end
    end
end
Fr_bin = Fr_bin ./ N

In [ ]:
heatmap(transpose(Fr_bin),aspect_ratio=:equal,c=:amp)

In [ ]:
heatmap(transpose(G2.weights),aspect_ratio=:equal,c=:amp)

In [ ]:
eps_edges = -8:1/64:-1
N2_edges = -1:1/64:1

for i=1:NZ
    global G
    h = fit(Histogram, (log10.(eps[:,:,i][:]), log10.(N2[:,:,i][:])), (eps_edges,N2_edges))
    if i==1
        G = h
    else
        merge!(G,h)
    end
end
G = normalize(G, mode=:pdf)

In [ ]:
N2=nothing
EK=nothing
GC.gc()